In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00


In [6]:
### Shortest Path Calculation ###
from torch_geometric.datasets import Planetoid, WebKB, WikipediaNetwork

DATASETS = {}

# Chamelion & Squirrel
# Cora & Citeseer
# Cornell & Texas & Wisconsin

## Mid Size Datasets
# Citation Networks
dataset = 'Cora'
dataset = Planetoid('/tmp/Cora', dataset)
data = dataset[0]
DATASETS['Cora'] = data
dataset = 'Citeseer'
dataset = Planetoid('/tmp/Citeseer', dataset)
data = dataset[0]
DATASETS['Citeseer'] = data
# Wikipedia Pages
dataset = 'Chameleon'
dataset = WikipediaNetwork(root='/tmp/Chameleon', name='Chameleon')
data = dataset[0]
DATASETS['Chameleon'] = data
dataset = 'Squirrel'
dataset = WikipediaNetwork(root='/tmp/Squirrel', name='Squirrel')
data = dataset[0]
DATASETS['Squirrel'] = data
### Small Sized Datasets
# Web Pages
dataset = WebKB(root='/tmp/Cornell', name='Cornell')
data = dataset[0]
DATASETS['Cornell'] = data
dataset = WebKB(root='/tmp/Texas', name='Texas')
data = dataset[0]
DATASETS['Texas'] = data
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin')
data = dataset[0]
DATASETS['Wisconsin'] = data


Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!


In [7]:
import tqdm
import networkx as nx
import torch
from torch_geometric.utils import remove_self_loops, dense_to_sparse, to_dense_batch, to_dense_adj

### Shortest Paths ###
def get_shortest_path_matrix(adjacency_matrix):
    graph = nx.from_numpy_array(adjacency_matrix.cpu().numpy(), create_using=nx.DiGraph)
    shortest_path_matrix = nx.floyd_warshall_numpy(graph)
    shortest_path_matrix = torch.tensor(shortest_path_matrix).float()
    return shortest_path_matrix

SHORTEST_PATHS = {}
for data_key in tqdm.tqdm(DATASETS):
  print(data_key)
  data = DATASETS[data_key]
  dense_adj = to_dense_adj(data.edge_index, max_num_nodes = data.x.shape[0])[0]
  dense_shortest_path_matrix = get_shortest_path_matrix(dense_adj)
  SHORTEST_PATHS[data_key] = dense_shortest_path_matrix

  0%|          | 0/7 [00:00<?, ?it/s]

Cora


 14%|█▍        | 1/7 [02:09<12:54, 129.05s/it]

Citeseer


 29%|██▊       | 2/7 [05:59<15:43, 188.76s/it]

Chameleon


 43%|████▎     | 3/7 [07:15<09:09, 137.36s/it]

Squirrel


100%|██████████| 7/7 [21:50<00:00, 187.15s/it]

Cornell
Texas
Wisconsin


In [11]:
### Save the Shortest Paths
import pickle

with open('sp_dict2.pkl', 'wb') as f:
    pickle.dump(SHORTEST_PATHS, f, protocol=pickle.HIGHEST_PROTOCOL)



In [12]:
import pickle
with open('sp_dict2.pkl', 'rb') as f:
    SHORTEST_PATHS_2 = pickle.load(f)

In [13]:
SHORTEST_PATHS_2

{'Cora': tensor([[0., 5., 4.,  ..., inf, 6., 5.],
         [5., 0., 1.,  ..., inf, 5., 5.],
         [4., 1., 0.,  ..., inf, 4., 4.],
         ...,
         [inf, inf, inf,  ..., 0., inf, inf],
         [6., 5., 4.,  ..., inf, 0., 1.],
         [5., 5., 4.,  ..., inf, 1., 0.]]),
 'Citeseer': tensor([[0., inf, inf,  ..., inf, inf, inf],
         [inf, 0., inf,  ..., 5., inf, 7.],
         [inf, inf, 0.,  ..., inf, inf, inf],
         ...,
         [inf, 5., inf,  ..., 0., inf, 9.],
         [inf, inf, inf,  ..., inf, 0., inf],
         [inf, 7., inf,  ..., 9., inf, 0.]]),
 'Chameleon': tensor([[0., 8., inf,  ..., 8., 6., inf],
         [inf, 0., inf,  ..., 6., 6., inf],
         [inf, 4., 0.,  ..., 4., 1., inf],
         ...,
         [inf, 9., inf,  ..., 0., 7., inf],
         [inf, 4., inf,  ..., 4., 0., inf],
         [inf, 6., inf,  ..., 6., 4., 0.]]),
 'Squirrel': tensor([[0., inf, 2.,  ..., 2., 2., 6.],
         [inf, 0., 4.,  ..., 4., 4., 7.],
         [inf, inf, 0.,  ..., 2., 3.